# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from cartopy import crs as ccrs
import plotly.express as px
import gmaps

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,laguna,38.4210,-121.4238,10.72,93,100,4.12,US,01-12-2022 23:17:18
1,1,hilo,19.7297,-155.0900,29.04,78,100,4.12,US,01-12-2022 23:17:18
2,2,sangar,63.9242,127.4739,-32.88,95,100,2.54,RU,01-12-2022 23:16:55
3,3,cukai,4.2500,103.4167,25.00,91,96,2.28,MY,01-12-2022 23:17:18
4,4,port elizabeth,-33.9180,25.5701,17.95,95,11,1.79,ZA,01-12-2022 23:13:21


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
mat_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map
mat_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values


temp_df = (city_data_df["Max Temp"]<27)& (city_data_df["Max Temp"]>21)
wind_df = (city_data_df["Wind Speed"]<4.5)
cloudiness_df = (city_data_df["Cloudiness"]==0)
# Drop any rows with null values

reduced_df = city_data_df.loc[(city_data_df["Max Temp"]<27)& (city_data_df["Max Temp"]>21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)]
reduced_df = reduced_df.dropna()
reduced_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
179,179,chatrapur,19.3500,84.9833,21.72,83,0,1.91,IN,01-12-2022 23:17:52
262,262,saint-pierre,-21.3393,55.4781,25.87,64,0,2.57,RE,01-12-2022 23:18:08
287,287,dwarka,22.2394,68.9678,23.08,55,0,2.92,IN,01-12-2022 23:18:12
466,466,abu dhabi,24.4667,54.3667,25.01,66,0,1.54,AE,01-12-2022 23:18:44
467,467,brandon,27.9378,-82.2859,22.71,70,0,2.24,US,01-12-2022 23:18:44
476,476,victor harbor,-35.5667,138.6167,21.49,71,0,3.52,AU,01-12-2022 23:18:45
515,515,acari,-15.4311,-74.6158,22.57,65,0,2.47,PE,01-12-2022 23:18:52
538,538,shakawe,-18.3667,21.8500,21.52,27,0,2.44,BW,01-12-2022 23:18:56
559,559,sinazongwe,-17.2614,27.4618,23.05,38,0,1.02,ZM,01-12-2022 23:19:01


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# hotel_df = reduced_df.copy()
# hotel_df = reduced_df.groupby("City").align["Country"], ["Lat"],["Lng"],["Humidity"]
# hotel_df

hotel_df = reduced_df.groupby(['City', 'Country', 'Lat', 'Lng', 'Humidity'], as_index=False)
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE


In [27]:
hotel_df = reduced_df

#Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 10
bias = f"proximity:{lng},{lat}"


# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row["City"]
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params= {
    "categories":categories,  
    "limit":limit,
    "filter":filter,
    "bias":bias,
    "apiKey":geoapify_key
   }
        
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
   
    # Convert the API response to JSON format
    name_address = name_address.json()
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chatrapur - nearest hotel: Gyan wine shop
saint-pierre - nearest hotel: Tropic Hotel
dwarka - nearest hotel: The Dwarika Hotel
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
brandon - nearest hotel: Brandon Motor Lodge
victor harbor - nearest hotel: Hotel Victor
acari - nearest hotel: No hotel found
shakawe - nearest hotel: Drotsky's Cabins
sinazongwe - nearest hotel: Lake View Lodge


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
179,179,chatrapur,19.3500,84.9833,21.72,83,0,1.91,IN,01-12-2022 23:17:52,Gyan wine shop
262,262,saint-pierre,-21.3393,55.4781,25.87,64,0,2.57,RE,01-12-2022 23:18:08,Tropic Hotel
287,287,dwarka,22.2394,68.9678,23.08,55,0,2.92,IN,01-12-2022 23:18:12,The Dwarika Hotel
466,466,abu dhabi,24.4667,54.3667,25.01,66,0,1.54,AE,01-12-2022 23:18:44,Marriott Downtown Abu-Dhabi
467,467,brandon,27.9378,-82.2859,22.71,70,0,2.24,US,01-12-2022 23:18:44,Brandon Motor Lodge
476,476,victor harbor,-35.5667,138.6167,21.49,71,0,3.52,AU,01-12-2022 23:18:45,Hotel Victor
515,515,acari,-15.4311,-74.6158,22.57,65,0,2.47,PE,01-12-2022 23:18:52,No hotel found
538,538,shakawe,-18.3667,21.8500,21.52,27,0,2.44,BW,01-12-2022 23:18:56,Drotsky's Cabins
559,559,sinazongwe,-17.2614,27.4618,23.05,38,0,1.02,ZM,01-12-2022 23:19:01,Lake View Lodge


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
mat_plot_2 = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "OSM",
frame_width = 700,
frame_height = 500,
size = "Hotel Name",
scale = 0.8,
color = "City_ID")

mat_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City_ID,Hotel Name)

In [ ]:
locations = hotel_df[["Lat", "Lng"]].astype(float)
hover_msg = gmaps.marker_layer(locations)

info_box = """

Hotel Name {Hotel Name}
City {City}
Country {Country}

"""
hotel_information = [info_box.format(**row) for index, row in hotel_df.iterrows()]
hotel_hover = gmaps.symbol_layer(
locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_information
)

fig = gmaps.figure()
fig.add_layer(hover_msg)
fig.add_layer(hotel_hover)
# Display the map

# Display the map
# YOUR CODE HERE